<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/Job%20Shop%20Scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Job Shop Scheduling Problem

Matematical model from: [Bruno Scalia C. F. Leite (2023)](https://towardsdatascience.com/the-job-shop-scheduling-problem-mixed-integer-programming-models-4bbee83d16ab)

## Notation

\begin{array}{ll}
M & \text{Set of machines}\\
J & \text{Set of jobs}\\
o^j  & \text{Processing order of job } j \text{ with } o^j = (o^j_1,o^j_2,o^j_3,...,o^j_{|M|}) \\
p_{ij} & \text{Processing time for job $i$ on machine $i$} \\
C & \text{Decision variable (integer) denoting the makespan (= objective)}\\
x_{mj} & \text{Decision variable (integer) denoting the time job $j$ starts on machine $m$}\\
z_{mij} & \text{Decision variable (binary): =1 if job $i$ is done before job $j$ on machine $m$, =0, else}\\
\end{array}

## Mathematical Model

$
\begin{align}
  \begin{array}{llll}
    &\textbf{Objective} & \\
    & \min C & &~~~  (1) \\
    &&\\
    &\textbf{Constraints} & \\
    & x_{o^j_{h-1}, j} + p_{o^j_{h-1}, j} \le x_{o^j_{h}, j} &,~ \forall~ j \in J, h \in (2,...,|M|) &~~~ (2) \\
    & x_{m, j} +  p_{m, j} \le  x_{m, k} + bigM \cdot (1 - z_{m,j,k}) &,~ \forall~ j,k \in J, m \in M: i \ne k &~~~ (3) \\
    & z_{m,j,k} + z_{m,k,j} = 1 &,~ \forall~ j,k \in J, m \in M: i \ne k &~~~ (4) \\
    &  x_{o^j_{|M|}, j} + p_{o^j_{|M|}, j} =\le C &,~ \forall~ j \in J &~~~ (5) \\
    &  x_{m,j} \ge 0 &,~ \forall~ j \in J, m \in M &~~~ (6) \\
    &  z_{m,j,k} \in \mathbb{B} &,~ \forall~ j,k \in J, m \in M &~~~ (7) \\
  \end{array}
\end{align}
$

## Python implementation

### Exemplary data

In [91]:
# set of machines
M = [1, 2, 3, 4]

# Set of jobs
J = [1, 2]

# [job] Ordered sequence of machines for a job
# IMPORTANT: add machines that are not used at the end of the sequence and set the corresponding processing time to zero
O = {1: [1, 3, 4] + [2],
     2: [1, 4, 2] + [3]}

# [machine][job] Processing times
p = {1: {1: 6, 2: 2},
     2: {1: 0, 2: 3},
     3: {1: 4, 2: 0},
     4: {1: 1, 2: 9}}

print('O: ' + str(O))
print('p: ' + str(p))


O: {1: [1, 3, 4, 2], 2: [1, 4, 2, 3]}
p: {1: {1: 6, 2: 2}, 2: {1: 0, 2: 3}, 3: {1: 4, 2: 0}, 4: {1: 1, 2: 9}}


### Packages

In [92]:
!pip install pulp
import pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Model

In [93]:
# Model
model = pulp.LpProblem(name='Job_Shop_Scheduling_Disjunct',
                       sense=pulp.constants.LpMinimize)

# Decision variables
C = pulp.LpVariable(name='C', lowBound=0, cat='Integer')
x = pulp.LpVariable.dicts(name='x', indices=(M, J), lowBound=1, cat='Integer')
z = pulp.LpVariable.dicts(name='z', indices=(M, J, J), cat='Binary')

# (1) Objective
model += C, '(1)'

# (2)
for j in J:
  for i in M:
    if i > 1:
      h = i-1 # list-index
      model += x[O[j][h-1]][j] + p[O[j][h-1]][j] <= x[O[j][h]][j], '(2)_' + str(j) + str(h)

# (3) und (4)
for j in J:
  for k in J:
    if j != k:
      for m in M:
        # (3)
        model += x[m][j] + p[m][j] <= x[m][k] + 9999 * (1 - z[m][j][k]), '(3)_' + str(m) + str(j) + str(k)

        # (4)
        model +=  z[m][j][k] +  z[m][k][j] == 1, '(4)_' + str(m) + str(j) + str(k)

# (5)
for j in J:
  model += x[O[j][-1]][j] + p[O[j][-1]][j] <= C, '(5)_' + str(j)

# print(model)

### Solution

In [94]:
# solve problem
model.solve()

# get status
print("Status:", pulp.LpStatus[model.status])

# get objective value
print('Objective value:', round(pulp.value(model.objective), 2))

# get value of decision variable u (position in tour of the nodes that are part of the tour)
for j in J:
  print('Job ' + str(j))
  for i in M:
    # if x[i][j].varValue > 0:
    print(' starts at machine ' + str(i) + ' at ' + str(x[i][j].varValue))

Status: Optimal
Objective value: 15.0
Job 1
 starts at machine 1 at 3.0
 starts at machine 2 at 15.0
 starts at machine 3 at 9.0
 starts at machine 4 at 13.0
Job 2
 starts at machine 1 at 1.0
 starts at machine 2 at 12.0
 starts at machine 3 at 15.0
 starts at machine 4 at 3.0
